<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/tft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook Overview




This notebook focuses on assessing the effectiveness of the **Temporal Fusion Transformer (TFT)** neural network model for our forecasting task.

Our approach includes:

- Applying the TFT model to our dataset.
- Conducting hyperparameter tuning to find the optimal model settings.
- Training the model exclusively on time series inputs — specifically using only the **store**, **department**, and **date** features — without incorporating additional external variables.

# Notebook Setup

The following setup is provided as a basic example for initializing the notebook environment. It includes necessary imports, optional configuration, and a placeholder for data loading or downloading.

This section is **not part of the core model logic**, and the code here may vary depending on your environment or data access method.

## Setup Environment


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

Cloning into 'Walmart-Recruiting-Store-Sales-Forecasting'...
remote: Enumerating objects: 462, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 462 (delta 31), reused 18 (delta 11), pack-reused 404 (from 1)
Receiving objects: 100% (462/462), 9.12 MiB | 13.17 MiB/s, done.
Resolving deltas: 100% (249/249), done.
/content/Walmart-Recruiting-Store-Sales-Forecasting


In [3]:
%%capture
!pip install -r requirements.txt

In [4]:
%%capture
from google.colab import userdata
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

In [5]:
from google.colab import userdata
wandb_api_key = userdata.get('WANDB_API_LOGIN')
!wandb login {wandb_api_key}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


## Load and Split Data

In [6]:
from src import data_loader, processing
import importlib
importlib.reload(processing)

dataframes = data_loader.load_raw_data()
df = processing.run_preprocessing(dataframes, process_test=False, merge_features=True, merge_stores=True)['train']
X_train, y_train, X_valid, y_valid = processing.split_data_by_ratio(df, separate_target=True)

print(f"Shapes of train_df and valid_df: {X_train.shape}, {X_valid.shape}")

Data loading complete.
Shapes of train_df and valid_df: (337256, 15), (84314, 15)


In [7]:
X_train

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,2,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2,1,3,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
3,1,4,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
4,1,5,2010-02-05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337251,22,27,2012-04-13,False,49.89,4.025,5981.5,10877.85,9.5,1633.96,1932.86,141.843393,7.671,B,119557
337252,22,28,2012-04-13,False,49.89,4.025,5981.5,10877.85,9.5,1633.96,1932.86,141.843393,7.671,B,119557
337253,22,29,2012-04-13,False,49.89,4.025,5981.5,10877.85,9.5,1633.96,1932.86,141.843393,7.671,B,119557
337254,22,30,2012-04-13,False,49.89,4.025,5981.5,10877.85,9.5,1633.96,1932.86,141.843393,7.671,B,119557


In [8]:
X_valid

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
337256,22,32,2012-04-13,False,49.89,4.025,5981.50,10877.85,9.5,1633.96,1932.86,141.843393,7.671,B,119557
337257,22,33,2012-04-13,False,49.89,4.025,5981.50,10877.85,9.5,1633.96,1932.86,141.843393,7.671,B,119557
337258,22,34,2012-04-13,False,49.89,4.025,5981.50,10877.85,9.5,1633.96,1932.86,141.843393,7.671,B,119557
337259,22,35,2012-04-13,False,49.89,4.025,5981.50,10877.85,9.5,1633.96,1932.86,141.843393,7.671,B,119557
337260,22,36,2012-04-13,False,49.89,4.025,5981.50,10877.85,9.5,1633.96,1932.86,141.843393,7.671,B,119557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,93,2012-10-26,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,B,118221
421566,45,94,2012-10-26,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,B,118221
421567,45,95,2012-10-26,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,B,118221
421568,45,97,2012-10-26,False,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,B,118221


# Train

We begin by defining the `run_patchtst_cv` method, which will be used throughout this notebook to perform cross-validation for the PatchTST model.

Note that the `NeuralForecastModels` framework is used and can be found in the `models/neural_forecast_models` directory.

In [9]:
from itertools import product
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT
from src.utils import wmae as compute_wmae

import logging
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("neuralforecast").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
logging.getLogger("lightning_fabric").setLevel(logging.WARNING)

def run_tft_cv(X_train, y_train, X_valid, y_valid,
                            param_grid,
                            fixed_params,
                            return_all=False):

    results = []

    keys, values = zip(*param_grid.items())
    for vals in product(*values):
        params = dict(zip(keys, vals))
        params.update(fixed_params)

        params['enable_progress_bar'] = False
        params['enable_model_summary'] = False

        model = TFT(**params)

        nf_model = NeuralForecastModels(models=[model], model_names=['TFT'], freq='W-FRI', one_model=True)
        nf_model.fit(X_train, y_train)
        y_pred = nf_model.predict(X_valid)
        score = compute_wmae(y_valid, y_pred, X_valid['IsHoliday'])

        result = {'wmae': score, 'preds': y_pred}
        result.update(params)

        results.append(result)
        print(" → ".join(f"{k}={v}" for k,v in params.items() if k not in ['enable_progress_bar','enable_model_summary']) + f" → WMAE={score:.4f}")

    if return_all:
        return results
    else:
        return min(results, key=lambda r: r['wmae'])


## Input Size and Batch Size




We will tune **input size** and **batch size** jointly because these two hyperparameters are closely related and impact training dynamics in complementary ways.

Tuning them together helps balance the amount of information the model sees per training step and the efficiency and stability of learning.

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from models.neural_forecast_models import NeuralForecastModels
from src.utils import wmae as compute_wmae

param_grid = {
    'input_size' : [36, 52, 60],
}

fixed_params = {
    'max_steps': 10 * 104,
    'h': 53,
    'random_seed': 42,
}


best_result = run_tft_cv(
    X_train, y_train, X_valid, y_valid,
    param_grid=param_grid,
    fixed_params=fixed_params,
    return_all=False
)

print("\nBest hyperparameters found:")
for param in param_grid.keys():
    print(f"  {param}: {best_result[param]}")
print(f"Best WMAE: {best_result['wmae']:.4f}")


input_size=36 → max_steps=1040 → h=53 → random_seed=42 → WMAE=1945.6900
input_size=52 → max_steps=1040 → h=53 → random_seed=42 → WMAE=1768.0978
input_size=60 → max_steps=1040 → h=53 → random_seed=42 → WMAE=1673.1952

Best hyperparameters found:
  input_size: 60
Best WMAE: 1673.1952


In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from models.neural_forecast_models import NeuralForecastModels
from src.utils import wmae as compute_wmae

param_grid = {
    'input_size' : [70, 80],
}

fixed_params = {
    'max_steps': 10 * 104,
    'h': 53,
    'random_seed': 42,
}


best_result = run_tft_cv(
    X_train, y_train, X_valid, y_valid,
    param_grid=param_grid,
    fixed_params=fixed_params,
    return_all=False
)

print("\nBest hyperparameters found:")
for param in param_grid.keys():
    print(f"  {param}: {best_result[param]}")
print(f"Best WMAE: {best_result['wmae']:.4f}")


input_size=70 → max_steps=1040 → h=53 → random_seed=42 → WMAE=1832.7233
input_size=80 → max_steps=1040 → h=53 → random_seed=42 → WMAE=1978.9080

Best hyperparameters found:
  input_size: 70
Best WMAE: 1832.7233


In [10]:
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from models.neural_forecast_models import NeuralForecastModels
from src.utils import wmae as compute_wmae

param_grid = {
    'batch_size' : [32, 64, 128],
}

fixed_params = {
    'max_steps': 10 * 104,
    'h': 53,
    'input_size': 60,
    'random_seed': 42,
}

best_result = run_tft_cv(
    X_train, y_train, X_valid, y_valid,
    param_grid=param_grid,
    fixed_params=fixed_params,
    return_all=False
)

print("\nBest hyperparameters found:")
for param in param_grid.keys():
    print(f"  {param}: {best_result[param]}")
print(f"Best WMAE: {best_result['wmae']:.4f}")

batch_size=32 → max_steps=1040 → h=53 → input_size=60 → random_seed=42 → WMAE=1673.1952
batch_size=64 → max_steps=1040 → h=53 → input_size=60 → random_seed=42 → WMAE=1701.3881
batch_size=128 → max_steps=1040 → h=53 → input_size=60 → random_seed=42 → WMAE=1701.0284

Best hyperparameters found:
  batch_size: 32
Best WMAE: 1673.1952


## Dropout




After tuning input size and batch size, we will focus on tuning **dropout**.Adjusting dropout after setting input and batch sizes allows us to fine-tune the regularization strength based on the chosen model complexity and training dynamics.

In [11]:
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from models.neural_forecast_models import NeuralForecastModels
from src.utils import wmae as compute_wmae

param_grid = {
    'dropout' : [0, 0.1, 0.2],
}

fixed_params = {
    'max_steps': 10 * 104,
    'h': 53,
    'input_size': 60,
    'random_seed': 42,
}

best_result = run_tft_cv(
    X_train, y_train, X_valid, y_valid,
    param_grid=param_grid,
    fixed_params=fixed_params,
    return_all=False
)

print("\nBest hyperparameters found:")
for param in param_grid.keys():
    print(f"  {param}: {best_result[param]}")
print(f"Best WMAE: {best_result['wmae']:.4f}")

dropout=0 → max_steps=1040 → h=53 → input_size=60 → random_seed=42 → WMAE=1756.2631
dropout=0.1 → max_steps=1040 → h=53 → input_size=60 → random_seed=42 → WMAE=1673.1952
dropout=0.2 → max_steps=1040 → h=53 → input_size=60 → random_seed=42 → WMAE=1749.9156

Best hyperparameters found:
  dropout: 0.1
Best WMAE: 1673.1952


In [12]:
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from models.neural_forecast_models import NeuralForecastModels
from src.utils import wmae as compute_wmae

param_grid = {
    'hidden_size': [64, 128],
}

fixed_params = {
    'max_steps': 10 * 104,
    'h': 53,
    'input_size': 60,
    'random_seed': 42,
    'dropout' : 0.1
}

best_result = run_tft_cv(
    X_train, y_train, X_valid, y_valid,
    param_grid=param_grid,
    fixed_params=fixed_params,
    return_all=False
)

print("\nBest hyperparameters found:")
for param in param_grid.keys():
    print(f"  {param}: {best_result[param]}")
print(f"Best WMAE: {best_result['wmae']:.4f}")

hidden_size=64 → max_steps=1040 → h=53 → input_size=60 → random_seed=42 → dropout=0.1 → WMAE=1792.6363
hidden_size=128 → max_steps=1040 → h=53 → input_size=60 → random_seed=42 → dropout=0.1 → WMAE=1673.1952

Best hyperparameters found:
  hidden_size: 128
Best WMAE: 1673.1952


## Optional Tuning of Optimizer




We will also tune the **optimizer** used during training, although this is not strictly necessary. Trying different optimizers can sometimes lead to improvements in training speed and model performance, but often the default choice works well.

In [13]:
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from models.neural_forecast_models import NeuralForecastModels
from src.utils import wmae as compute_wmae
import torch.optim as optim

param_grid = {
    'optimizer': [optim.Adam, optim.RMSprop],
}

fixed_params = {
    'max_steps': 10 * 104,
    'h': 53,
    'input_size': 60,
    'random_seed': 42,
    'dropout' : 0.1
}

best_result = run_tft_cv(
    X_train, y_train, X_valid, y_valid,
    param_grid=param_grid,
    fixed_params=fixed_params,
    return_all=False
)

print("\nBest hyperparameters found:")
for param in param_grid.keys():
    print(f"  {param}: {best_result[param]}")
print(f"Best WMAE: {best_result['wmae']:.4f}")

optimizer=<class 'torch.optim.adam.Adam'> → max_steps=1040 → h=53 → input_size=60 → random_seed=42 → dropout=0.1 → WMAE=1673.1952
optimizer=<class 'torch.optim.rmsprop.RMSprop'> → max_steps=1040 → h=53 → input_size=60 → random_seed=42 → dropout=0.1 → WMAE=1733.5558

Best hyperparameters found:
  optimizer: <class 'torch.optim.adam.Adam'>
Best WMAE: 1673.1952


# Best Model


The best TFT model selected through cross-validation has the following parameters:

| Parameter               | Value          |
|-------------------------|----------------|
| **input_size**          | 60             |
| **dropout**             | 0.1            |
| **h (forecast horizon)**| 53             |
| **max_steps**           | 2080 (20 * 104)|
| **random_seed**         | 42             |


In [15]:
from neuralforecast.models import TFT
from models.neural_forecast_models import NeuralForecastModels
from src.utils import wmae as compute_wmae

model = TFT(
    input_size=60,
    dropout = 0.1,
    h=53,
    max_steps= 20 * 104,
    random_seed=42,
    enable_progress_bar = False,
    enable_model_summary = False
)
nf_model = NeuralForecastModels(models=[model], model_names=['TFT'], freq='W-FRI', one_model=True)
nf_model.fit(X_train, y_train)
y_pred = nf_model.predict(X_valid)
wmae = compute_wmae(y_valid, y_pred, X_valid['IsHoliday'])

print(wmae)

1717.1530206837165


Now, we will train the selected best model on the entire dataset to leverage all available data. Additionally, we will log the training process and metrics using **Weights & Biases (wandb)** for experiment tracking.

In [16]:
from neuralforecast.models import TFT
from models.neural_forecast_models import NeuralForecastModels
from src.utils import wmae as compute_wmae

model = TFT(
    input_size=60,
    dropout = 0.1,
    h=53,
    max_steps= 20 * 104,
    random_seed=42,
    enable_progress_bar = False,
    enable_model_summary = False
)
nf_model = NeuralForecastModels(models=[model], model_names=['TFT'], freq='W-FRI', one_model=True)
nf_model.fit(df.drop(columns='Weekly_Sales'), df['Weekly_Sales'])

In [17]:
from configs import basic_config
from configs import nn_models_config

import importlib
importlib.reload(basic_config)
importlib.reload(nn_models_config)

from sklearn.pipeline import Pipeline
from configs.basic_config import minimal_config as cfg
from configs.nn_models_config import tft_config
from src.utils import log_to_wandb

log_to_wandb(
    model=nf_model,
    train_score=-1,
    val_score=wmae,
    config=cfg | tft_config,
    run_name='tft_02',
    artifact_name="tft",
)

wandb: Currently logged in as: zhorzholianimate (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


train_wmae,▁
val_wmae,▁
train_wmae,-1
val_wmae,1717.15302
